In [5]:
import json

secret = {"username":"abhay1704","key":"c29c06b6ec949fc93f5a72b99420ff2a"}
json.dump(secret, open('/root/.config/kaggle/kaggle.json', 'w'))

!kaggle kernels output abhay1704/include-finetuned -p /kaggle/working/

Output file downloaded to /kaggle/working/timesformer-finetuned/checkpoint-13149/config.json
Output file downloaded to /kaggle/working/timesformer-finetuned/checkpoint-13149/model.safetensors
Output file downloaded to /kaggle/working/timesformer-finetuned/checkpoint-13149/optimizer.pt
Output file downloaded to /kaggle/working/timesformer-finetuned/checkpoint-13149/preprocessor_config.json
Output file downloaded to /kaggle/working/timesformer-finetuned/checkpoint-13149/rng_state.pth
Output file downloaded to /kaggle/working/timesformer-finetuned/checkpoint-13149/scheduler.pt
Output file downloaded to /kaggle/working/timesformer-finetuned/checkpoint-13149/trainer_state.json
Output file downloaded to /kaggle/working/timesformer-finetuned/checkpoint-13149/training_args.bin
Output file downloaded to /kaggle/working/timesformer-finetuned/checkpoint-14610/config.json
Output file downloaded to /kaggle/working/timesformer-finetuned/checkpoint-14610/model.safetensors
Output file downloaded to /k

In [20]:
from transformers import AutoImageProcessor, TimesformerForVideoClassification
import cv2
import torch

processor = AutoImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400")
idx_to_class = json.load(open('/kaggle/working/timesformer-finetuned/idx_to_class.json', 'r'))
class_to_idx = {y:x for x,y in idx_to_class.items()}

model = TimesformerForVideoClassification.from_pretrained(
    '/kaggle/working/timesformer-finetuned',
    num_labels=len(class_to_idx),  
    ignore_mismatched_sizes=True  
)


# Step 7: Inference Function
def predict_video(video_path):
    # Load video frames
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Error: Could not open video file.")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_step = max(total_frames // 8, 1)

    video_frames = []
    for i in range(8):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_step)
        ret, frame = cap.read()
        if ret:
            # Resize frame to 224x224 and convert to RGB
            frame_resized = cv2.resize(frame, (224, 224))
            frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
            video_frames.append(frame_rgb)
        else:
            break

    cap.release()

    # Process frames
    inputs = processor(images=[video_frames], return_tensors="pt").to(model.device)

    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_idx = logits.argmax(-1).item()

    # Map index to class name
    idx_to_class = {idx: cls for cls, idx in class_to_idx.items()}
    predicted_class = idx_to_class[str(predicted_class_idx)]

    print(f"Predicted class: {predicted_class}")



In [19]:
# idx_to_class

In [21]:
predict_video('/kaggle/input/include/Adjectives_2of8/Adjectives/11. rich/MVI_9596.MOV')

Predicted class: rich


In [22]:
predict_video('/kaggle/input/include/Home_3of4/Home/40. Paint/MVI_4413.MOV')

Predicted class: paint


In [23]:
predict_video('/kaggle/input/include/People_5of5/People/80. Adult/MVI_3825.MOV')

Predicted class: adult
